In [11]:
import os
import glob
import json
import pymongo
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool

In [12]:
def CreateDB(db_name):
    try:
        postgres_con = psycopg2.connect(database="postgres",user="karimkhalifa")
        postgres_con.autocommit = True
        cursor = postgres_con.cursor()
        res = cursor.execute(f'''CREATE database {db_name}''')
        postgres_con.close()
    except:
        print("Postgres database exists") 
    mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
    engine = create_engine(f"postgres:///{db_name}",poolclass=NullPool)
    return (db_name,engine,mongo_client)

In [13]:
def CreateTables(db_name,engine,mongo_client,files):
    for f in files:
        table_name = f.rsplit("-",1)[1].replace(".csv","").strip()
        df = pd.read_csv(f)
        if "Mongo" in f:
            InsertMongo(db_name,mongo_client,df,table_name)
        elif "Postgres" in f:
            try:
                df.to_sql(table_name, engine, index=False)
            except:
                print(f"Table {table_name} already exists")

In [14]:
def InsertMongo(db_name,myclient,df,table_name):
    mydb = myclient[db_name]
    collection = mydb[table_name]
    records = json.loads(df.T.to_json()).values()
    for record in records:
        existing_record = collection.find_one(record)
        if not existing_record:
            collection.insert_one(record)
        else:
            collection.replace_one(existing_record,record,upsert=True)

In [15]:
if __name__ == "__main__":
    db_name = "packform"
    try:
        csvs = glob.glob(os.path.join(os.getcwd(),"test_data/*.csv"))
        CreateTables(*CreateDB(db_name),csvs)
        print("Finished successfully...")
    except:
        print("An error has occured!")

Finished successfully...
